In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# Train on these
normal_market_path = "training_data_normal_market.parquet"
stressed_market_path = "training_data_stressed_market.parquet"
hft_dominated_path = "training_data_hft_dominated.parquet"

# Just to visualize
mini_flash_crash = "training_data_mini_flash_crash.parquet"
flash_crash = "training_data_flash_crash.parquet"

# Concatting the files that we will actually train on
files = [
    normal_market_path,
    stressed_market_path,
    hft_dominated_path,
]

df = pd.concat([pd.read_csv(f) for f in files])

### Visualizing the data

In [ ]:
normal_market_df = pd.read_parquet(normal_market_path)

print(normal_market_df.head())

In [ ]:
# Scenarios to labels
le = LabelEncoder()
df["label"] = le.fit_transform(df["scenario"])

X = df.drop(columns=["scenario", "label"])
y = df["label"]

In [ ]:
# XGBoost 
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=42, stratify=y
)

model = XGBClassifier(
    n_estimators=300,
    max_depth=5,
    learning_rate=0.05,
    subsample=0.8,
    colsample_bytree=0.8,
    objective="multi:softmax",
    num_class=5
)

model.fit(X_train, y_train)

y_pred = model.predict(X_test)

print(classification_report(y_test, y_pred, target_names=le.classes_))